In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import os
from PIL import Image

In [8]:
import os
# Määritä ZIP-tiedoston nimi ja kohdekansio
zip_tiedoston_nimi = 'syopa.zip'  # Korvaa 'nimesi.zip' ZIP-tiedostosi nimellä
kohdekansio = '.'  # Tämä tarkoittaa nykyistä kansiota Jupyterissa

with zipfile.ZipFile(zip_tiedoston_nimi, 'r') as zip_ref:
    zip_ref.extractall(kohdekansio)

print(f"'{zip_tiedoston_nimi}' on purettu onnistuneesti.")

#tarkista tiedostorakenne: Kuten aiemmin, voit tarkistaa, onko tiedostorakenne kuten odotit: python
# Tarkista, onko tiedostorakenne oikein
for dataset in ['train', 'test', 'val']:
    for category in ['benign', 'malignant']:
        polku = os.path.join(kohdekansio, 'output', dataset, category)
        if os.path.exists(polku):
            print(f"Alikansio {dataset}/{category} on olemassa.")
        else:
            print(f"Alikansiota {dataset}/{category} ei löydy!")


'syopa.zip' on purettu onnistuneesti.
Alikansio train/benign on olemassa.
Alikansio train/malignant on olemassa.
Alikansio test/benign on olemassa.
Alikansio test/malignant on olemassa.
Alikansio val/benign on olemassa.
Alikansio val/malignant on olemassa.


In [9]:

def esikasittele_kuvat(kuvat, koko=(150, 150)):
    esikasitellyt = []
    for kuva in kuvat:
        kuva = kuva.resize(koko)
        kuva_array = tf.keras.preprocessing.image.img_to_array(kuva)
        esikasitellyt.append(kuva_array)
    return esikasitellyt

train_benign_kuvat_esikasitelty = esikasittele_kuvat(train_benign_kuvat)

NameError: name 'train_benign_kuvat' is not defined

In [46]:

# Here we load the images and do some preprocessing for them. We do the following preprocessing steps:
# - transform the into arrays
# - resize them to a smaller resolution
# - convert the image to B&W
# - Normalize the arrays

def transform_image(kuva):

    kuva = Image.open(kuva).convert('L')
    new_size = (28, 28)

    resized_image = kuva.resize(new_size, Image.BILINEAR)

    kuva_array = tf.keras.preprocessing.image.img_to_array(resized_image)
    normalized_array = (kuva_array - kuva_array.min()) / (kuva_array.max() - kuva_array.min())

    return normalized_array


# Iterating through the folders to apply preprocessing for all images in the dataset 

def go_through_folder(folder):
    array_list =[]
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        array = transform_image(file_path)
        array_list.append(array)
        
    return array_list
        
#test_image = transform_image("output/train/benign/SOB_B_A-14-22549AB-400-001.png")   
benign_train = np.array(go_through_folder("output/train/benign"))
malignant_train = np.array(go_through_folder("output/train/malignant"))
benign_test = np.array(go_through_folder("output/test/benign"))
malignant_test = np.array(go_through_folder("output/test/malignant"))

benign_train[0]


array([[[0.33566433],
        [0.53846157],
        [0.34265736],
        [0.34265736],
        [0.30769232],
        [0.15384616],
        [0.41958043],
        [0.74825174],
        [0.6923077 ],
        [0.8181818 ],
        [0.5874126 ],
        [0.54545456],
        [0.46153846],
        [0.62237763],
        [0.67132866],
        [0.43356642],
        [0.45454547],
        [0.46853146],
        [0.28671327],
        [0.2937063 ],
        [0.4125874 ],
        [0.8321678 ],
        [0.97902095],
        [0.97202796],
        [0.96503496],
        [0.95804197],
        [0.83916086],
        [0.6013986 ]],

       [[0.41958043],
        [0.5034965 ],
        [0.33566433],
        [0.43356642],
        [0.4055944 ],
        [0.30769232],
        [0.45454547],
        [0.57342654],
        [0.5174825 ],
        [0.6853147 ],
        [0.37762237],
        [0.34965035],
        [0.41958043],
        [0.45454547],
        [0.41958043],
        [0.3986014 ],
        [0.4755245 ],
        

In [47]:
# Creating the Y labels for their respective images, 1 being a malignant tumor, 0 being a benign tumor
# Also preparing them to their respective train and test sets
benign_train_label = np.zeros(len(benign_train))
benign_test_label = np.zeros(len(benign_test))

malignant_train_label = np.ones(len(malignant_train))
malignant_test_label = np.ones(len(malignant_test))


X_train = np.concatenate((benign_train, malignant_train), axis = 0)
Y_train = np.concatenate((benign_train_label, malignant_train_label), axis = 0)


X_test = np.concatenate((benign_test, malignant_test), axis = 0)
Y_test = np.concatenate((benign_test_label, malignant_test_label), axis = 0)
type(X_test)

numpy.ndarray

In [48]:
# Creating the neural network

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 128)         147584    
                                                                 
 flatten_1 (Flatten)         (None, 1152)             

In [49]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', 'loss'])

history = model.fit(X_train, Y_train, epochs = 10, batch_size = 64, shuffle = True,
                    validation_data=(X_test, Y_test))

Epoch 1/10


TypeError: in user code:

    File "/Users/ahtiholli/venv-metal/lib/python3.9/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/ahtiholli/venv-metal/lib/python3.9/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ahtiholli/venv-metal/lib/python3.9/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/ahtiholli/venv-metal/lib/python3.9/site-packages/keras/src/engine/training.py", line 1085, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/ahtiholli/venv-metal/lib/python3.9/site-packages/keras/src/engine/training.py", line 1179, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/Users/ahtiholli/venv-metal/lib/python3.9